# DS3: Solving the three-body problem

Fill in this notebook in the spaces provided to solve the three-body problem. The fundamental method is exactly the same as we did in the classes with the harmonic oscillator or the one-body problem. 

In [ ]:
from vpython import *
scene = canvas()                 # This line makes the output show below this cell

# Setting up the stage for Visual Python

In this part we'll begin by setting up the stage. In this first cell below, we will define all the physical constants that we need. 

In [ ]:
# Values of physical constants that we will need

G=6.673e-11                       # Defining Newton's Gravitational Constant
myPi = 3.141592                   # Value of pi used
sun_mass = 2e30                   # The mass of the sun in kilograms
earth_mass = 6e24                 # The mass of the earth in kilograms

boost=1.0                         # Allow for increasing Jupiter's mass by a factor

jupiter_mass=boost*1.9e27         # Jupiter's mass

# Values required for initial conditions

AU = 149.6e9                      # The mean earth-sun orbital distance

earth_vel  = 2 * myPi * 1  *AU/(365.25 *24. *60.*60.)    # Average velocity of the Earth = 2*Pi*R/T
jupiter_vel= 2 * myPi * 5.2*AU/(11.86*365.25*24.*60.*60) # Average velocity of Jupiter

### Defining the initial conditions

As we've discussed in class, we need to specify the initial positions and initial velocities of all the objects. In this program, let us assume that the sun is initially at 'rest', and at the origin.

`sun_pos_vector = vector(0,0,0)`

`sun_vel_vector = vector(0,0,0)`

We'll set the Earth and Jupiter to their associated distances from the sun, and let's assume that their initial velocities are only in the $y$ direction:

`earth_pos_vector = vector(AU,0,0)`

`earth_vel_vector = vector(0,earth_vel,0)`

`jupiter_pos_vector = vector(5.2*AU,0,0)`

`jupiter_vel_vector = vector(0,jupiter_vel,0)`

In [ ]:
# Defining the initial conditions as vectors

sun_pos_vector = vector(0,0,0)
sun_vel_vector = vector(0,0,0)

earth_pos_vector = vector(AU,0,0)
earth_vel_vector = vector(0,earth_vel,0)

jupiter_pos_vector = vector(5.2*AU,0,0)
jupiter_vel_vector = vector(0,jupiter_vel,0)

### Defining VPython objects

Let us now go one step further and define the scene. This consists of a background (called the `scene`) which can have properties like colour and a width (or `range`), and a set of 3D structures likes `box`es, `sphere`s, `cone`s and so on. You can find out more information about them [here](https://www.glowscript.org/docs/VPythonDocs/index.html). There are also some short [instructional videos on Youtube](https://www.youtube.com/user/VPythonVideos/videos) which could be very helpful.

We will be defining the objects in our 'solar system' (the Sun, Earth, and Jupiter) as being `sphere`s. A VPython sphere requires some information to be defined. It needs to be given 
1. A position or `pos`, which is a vpython `vector` object, 
2. A `radius`, which is simply a number. (If this is not given, it sets a default radius.)
3. A `color`, which is a vpython color object (given by `color.{color_name}`)

Now, we can also define other attributes that this `sphere` possesses. In particular, we could define a `velocity` and `mass`, even though none of these quantities are predefined. These will not affect what the sphere does or looks like, they are simply numbers (or vectors) which can be called by `{sphere_name}.mass` or `{sphere_name}.velocity`.

Once you have finished reading and understanding the next cell, run all the cells up to it. When you're done, you should see an image above the **first cell** of this notebook which is a (static) solar system!



In [ ]:
# Setting up the background.

scene.background = color.white          # Setting the background colour
scene.autoscale = False                 # Don't scale the scene size automatically
scene.range = 10*AU                     # The extent of the region of interest to the left or right of center.

# Defining the objects that make up our 'solar system'

sun = sphere(pos = vector(0,0,0),       # The sun is placed at the origin
             velocity = vector(0,0,0),  # A new variable (you could just call this v)
             mass=sun_mass,             # A new variable (you could just call this m)
             radius = 0.1*AU,           # The radius of the sun (NOT ACCURATE!)
             color =color.yellow)       # The colour of the sun (... what do _you_ think?)

# We now do exactly the same thing for the Earth and Jupiter below

earth = sphere(pos= vector(AU, 0, 0), velocity = vector(0,earth_vel,0), mass=earth_mass, radius=0.05*AU, color =color.cyan)
jupiter=sphere(pos=vector(5.2*AU,0,0),velocity=vector(0,jupiter_vel,0), mass=jupiter_mass, radius=0.15*AU, color=color.red)

# Note:  The radius of sun, jupiter  and earth are NOT their true radii! 
#        These are the radii of the spherical object that will be drawn 
#        on the computer screen.

# Exercise 1: Adding new properties to VPython objects

Let us now do two simple things: we'll begin by creating a list of objects that comprise our solar system, and we will assign to **each** of them a *new* variable, called `acc` which is a `vector`, and which is initially zero in all directions.

We will also define a variable called `track` which is a curve of the same colour as the object that will record the trajectory of the moving object.

In [ ]:
#Create a list of objects making up our solar system 
#and add attributes for their accelaration and orbits

bodies =                           # <=== Create a list of objects in our solar system

for b in bodies:                   # For each object in `bodies`, "b",
    b.acc =                        # <=== Assign a variable called `acc` which is a (zero) vector
    b.track=curve(color = b.color) # <=== Assign another variable called track which is a curve with the same colour

# Exercise 2: Setting up the dynamics

In this part of the exercise, you will define the dynamics of the system, using Newton's Law of Universal Gravitation. The law says that the gravitational force on an object of mass $m_A$ is given by

$$F_\text{grav} = \frac{G m_A m_B}{|r_{AB}|^2} \hat{r},$$

where $r$ is the relative position between the two objects, $A$ and $B$, $\vec{r} = \vec{r_A}-\vec{r_B}.$ Now, from Newton's Second Law of motion, we know that the force on the mass $A$ is given by 

$$\vec{F}_A = m_A \times \vec{a}_A,$$ 

and therefore the acceleration that the mass $A$ experiences because of $B$ is: $$\vec{a}_A = \frac{G m_B}{|r_{AB}|^2}\hat{r}.$$

Now what about the acceleration that the mass $B$ experiences because of $A$? The force on $B$ due to $A$ is the same as before:

$$F_\text{grav} = \frac{G m_A m_B}{|r_\text{BA}|^2} \hat{r},$$

the difference is that the *acceleration* of $B$ due to $A$ is: $$\vec{a}_B = \frac{G m_A}{|r_\text{BA}|^2}\hat{r}.$$

(Make sure you understand this sufficiently clearly.)

## The net acceleration on an object

Suppose now you have a bunch of objects ($A,B,C,\dots$) all interacting with each other. Since force is a vector, the net force acting on the object $A$ due to all the others is: $$F_\text{net} = \vec{F}_{AB} + \vec{F}_{AC} + \vec{F}_{AD}\dots$$

Thus, the acceleration that $A$ experiences is 

$$a_A = \frac{\vec{F}_{AB}}{m_A} + \frac{\vec{F}_{AB}}{m_A} + \dots.$$

Using the form of the gravitational force law, we can see that 

$$a_A = \frac{G m_B}{r_{AB}^2} + \frac{G m_C}{r_{AC}^2} + \dots, $$

or in other words, you need to add the individual accelerations due to each of the other objects in the solar system. This is an important point, make sure you understand it.

In the cell below, the function `acc(a,b)` accepts two vpython objects `a` and `b` calculates the acceleration of the object `a` due to the object `b`. The `totalacc(a,objlist)` function accepts a vpython object `a` and a list `objlist` and calculates the acceleration on `a` due to all the objects in `objlist` (except `a`).

Some useful functions are: 
- `{vector_name}.mag` which returns the magnitude of a vpython vector
- `{vector_name}.mag2` which returns the magnitude squared of a vpython vector
- `norm({vector_name})` which returns a unit vector in the direction of the vector

In [ ]:
def acc(a, b):  # This function calculates the acceleration of an object a due
                # to an object b. This function accepts two vpython variables.
    
    rel_pos =   # <=== Find the relative position of a and b. (You can use the 
                #      fact that the position of a can be given by a.pos, which
                #      returns a vpython array <x,y,z>.
    
    g_a =       # <=== Now, use Newton's gravitational law to calculate the acceleration
                #      of a, using the formula.
    
    return g_a  #  Returns a vpython array of acceleration. G*b.mass * norm(rel_pos)/rel_pos.mag2


# We will now define a function that calculates the total acceleration on an object
# a, due to all the other objects that are *not* a.

def totalacc(a, objlist):
    sum_acc = vector(0,0,0)                # The total acceleration is intitially given a vector value of <0,0,0>
    
    for b in objlist:                      # For every element in the list of objects...
        if (a!=b):                         # As long as the object is not a
            sum_acc = sum_acc + acc(a, b)  # Find a's acceleration due to the object, and add it to the sum.
    return sum_acc                         # When this is done, return the total acceleration.

## Going to the centre of mass frame

It turns out that the centre of mass frame is much easier to actually study this system, as in this frame the total momentum of system is zero. As we began by assuming that the sun was at rest and the planets moving around it, going to the centre of mass frame simply requires us to calculate the total momentum of the Earth and Jupiter, and to give the sun the negative of that momentum, so that the total momentum of the system is zero.

This may not seem like a big deal to the Sun-Earth-Jupiter system since the centre of mass of this system is very close to the centre of the sun, but it will certainly be important if you considered -- for example -- a binary star system.

In [ ]:
sum_momentum = vector(0,0,0)

for b in bodies:
    if (b!=sun):
        sum_momentum = sum_momentum +b.mass*b.velocity

sun.velocity = - sum_momentum/sun.mass

## Setting the time-step

This is pretty self-evident, I hope.

In [ ]:
dt = 3000.*60.      # The time-step dt corresponds to 3000 mins here

# Exercise 3: Solving the Kinematics

This last part of the code involves the actual mechanics of solving the equation. We begin by initialising the velocities of each of the objects as done in the leap-frog method.

We then begin an infinite loop which updates the positions of each of the bodies depending on their instantaneous velocities, and also the velocities of each of the bodies depending on their instantaneous accelerations. The accelerations are calculated by the function we defined earlier, and uses the instantaneous positions of all the objects. Spend some time looking at this and trying to understand it.

In [ ]:
# Let's begin by initialising the leap-frog by finding the velocites at t=dt/2

for b in bodies:       
    b.velocity = # <=== Initialise the velocity of each element `b` in the 
                 #      list `bodies` using the leap-frog method

# We are now ready to start the leap-frog method

while(True):     # Running an infinite loop
    rate(50)     # This limits the number of loops in a second (in this case, 50)
    
    for b in bodies:
        # Fill in this loop
                                 # <=== Update the positions
                                 # <=== Update the velocities
        b.track.append(pos=b.pos)# This line updates the tracking curve's position 
        

    scene.center = vector(0,0,0) # This centres the view on the origin of the coordinate system after every loop.
